### 描述子（Descriptors）

描述子是 Python 中一個強大的特性，允許您自定義類別中的**屬性訪問行為**。描述子在 Python 中對於管理屬性訪問和封裝行為至關重要，沒有它們，實現像驗證、延遲載入和自定義存儲等功能將變得顯著更複雜和難以處理。

### 什麼是描述子？

描述子是**定義屬性訪問方式**的物件。它們通常是通過定義一個或多個以下方法的類別來實現的：

- `__get__(self, instance, owner)`：當訪問屬性時，此方法被呼叫。
- `__set__(self, instance, value)`：當設置屬性時，此方法被呼叫。
- `__delete__(self, instance)`：當刪除屬性時，此方法被呼叫。

### 何時使用描述子？

當描述子類別的實例出現在另一個類別的類別字典中（稱為“擁有者類別”）時，描述子會被使用。這意味著描述子必須是擁有者類別或其父類別的屬性。

### 描述子的關鍵方法

1. **`object.__get__(self, instance, owner=None)`**：
   - **目的**：當訪問描述子型態的(擁有者類別之)屬性時，將呼叫此方法。
   - **參數**：
     - `instance`：訪問描述子實體時的擁有者類別的實例。如果直接從類別訪問（例如，`OwnerClass.attribute`），則可以是 `None`。
     - `owner`：擁有此描述子的擁有者類別。此參數是可選的，指示哪個類別擁有描述子。
   - **返回值**：此方法應返回計算的屬性值，或者在無法檢索屬性時引發 `AttributeError`。
   - **示例用法**：這對於動態計算值或控制屬性的訪問非常有用。

2. **`object.__set__(self, instance, value)`**：
   - **目的**：當將新值設置為擁有者類別的實例屬性時，將呼叫此方法。
   - **參數**：
     - `instance`：要設置屬性的擁有者類別的實例。
     - `value`：要分配給屬性的新的值。
   - **註解**：通過定義 `__set__`，描述子變成“數據描述子”。這意味著它不僅管理獲取值，還管理設置值。

3. **`object.__delete__(self, instance)`**：
   - **目的**：當從擁有者類別的實例中刪除屬性時，將呼叫此方法。
   - **參數**：
     - `instance`：要刪除屬性的實例。
   - **用法**：這允許控制如何從實例中刪除屬性。

### 描述子的示例

```python
class Descriptor:
    def __init__(self):
        self.value = None

    def __get__(self, instance, owner):
        if instance is None:
            return self  # 通過擁有者類別訪問時, 回傳描述子實體本身
        return self.value  # 返回存儲的值

    def __set__(self, instance, value):
        self.value = value  # 設置值

    def __delete__(self, instance):
        del self.value  # 刪除值

class Owner:
    attr = Descriptor()  # 創建描述子實例

# 用法
owner_instance = Owner()
owner_instance.attr = 42  # 呼叫 Descriptor.__set__()
print(owner_instance.attr)  # 呼叫 Descriptor.__get__()，輸出: 42
del owner_instance.attr  # 呼叫 Descriptor.__delete__()
```

### 重要說明

- **數據描述子與非數據描述子**：
  - **數據描述子**：定義了 `__get__` 和 `__set__`（和/或 `__delete__`）。它們優先於實例屬性。
  - **非數據描述子**：僅定義 `__get__`。它們不管理設置或刪除屬性。
 
### 結論

描述子是 Python 中一個強大的特性，可以精細地控制如何訪問和修改類別中的屬性。通過實現 `__get__`、`__set__` 和 `__delete__` 方法，您可以為屬性訪問定義自定義行為，使您的類別更加靈活和表達力強。這一能力特別適合用於實現屬性、管理狀態和強制封裝。

### 描述子應用場景

描述子在需要以更受控或動態的方式管理屬性訪問的情況下特別有用。以下是幾個描述子可以至關重要的場景：

### 1. **帶有驗證的屬性(Properties with Validation)**

我們希望確保某個屬性只能設置為有效值。使用描述子可以將這一邏輯整齊地封裝起來。

#### 示例：驗證範圍

```python
class PositiveInteger:
    def __init__(self):
        self.value = 0

    def __get__(self, instance, owner):
        return self.value

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("必須是正整數")
        self.value = value

class Person:
    age = PositiveInteger()

# 用法
p = Person()
p.age = 30  # 有效
print(p.age)  # 輸出: 30

try:
    p.age = -5  # 引發 ValueError
except ValueError as e:
    print(e)  # 輸出: 必須是正整數
```

### 2. **延遲載入(Lazy Loading)**

描述子可以用於屬性的延遲載入，當實際需要時才計算其值。

#### 示例：屬性的延遲計算

```python
class LazyAttribute:
    def __init__(self, compute_func):
        self.compute_func = compute_func
        self.value = None

    def __get__(self, instance, owner):
        if self.value is None:
            self.value = self.compute_func(instance)
        return self.value

class Data:
    def __init__(self, data):
        self.data = data

    @LazyAttribute
    def processed_data(self):
        print("處理數據...")
        return sum(self.data)  # 模擬耗時操作

    # Above code is equivalent to below
    # processed_data = LazyAttribute(lambda instance: sum(instance.data))


# 用法
d = Data([1, 2, 3, 4])
print(d.processed_data)  # 輸出: 處理數據... 10
print(d.processed_data)  # 輸出: 10（這次不再處理）
```

### 3. **管理訪問控制(Managing Access Control)**

描述子還可以用來管理對某些屬性的訪問控制，例如使某些屬性為唯讀。

#### 示例：唯讀屬性

```python
class ReadOnly:
    def __init__(self, value):
        self.value = value

    def __get__(self, instance, owner):
        return self.value

    def __set__(self, instance, value):
        raise AttributeError("此屬性為只讀")

class User:
    username = ReadOnly("user123")

# 用法
u = User()
print(u.username)  # 輸出: user123

try:
    u.username = "new_user"  # 引發 AttributeError
except AttributeError as e:
    print(e)  # 輸出: 此屬性為只讀
```

### 4. **自定義屬性存儲方式(Customizing Attribute Storage)**

描述子允許您管理屬性的存儲方式，這對於實現緩存(caching)或自定義存儲後端(custom storage backends)等功能非常有用。

#### 示例：屬性的自定義存儲

```python
class JSONStorage:
    def __init__(self, filename):
        self.filename = filename
        self._data = self.load()

    def load(self):
        import json
        try:
            with open(self.filename) as f:
                return json.load(f)
        except FileNotFoundError:
            return {}  # Return an empty dictionary if the file does not exist

    def __get__(self, instance, owner):
        return self._data.get(instance.id)

    def __set__(self, instance, value):
        self._data[instance.id] = value
        self.save()

    def save(self):
        import json
        with open(self.filename, 'w') as f:
            json.dump(self._data, f)

class User:
    storage = JSONStorage('user_data.json')
    
    def __init__(self, id):
        self.id = id

# 用法
user1 = User('1') # note the id is a str
print(user1.storage)  # return None if not exist
user1.storage = {"name": "Steve", "age": 30}  # 保存到 JSON 文件
print(user1.storage)  # 從 JSON 文件檢索數據
#
user2 = User('2')
print(user2.storage)  # return None if not exist
user2.storage = {"name": "John", "age": 20}  # 保存到 JSON 文件
print(user2.storage)  # 從 JSON 文件檢索數據
```

In [7]:
class PositiveInteger:
    def __init__(self):
        self.value = 0

    def __get__(self, instance, owner):
        return self.value

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Must be a positive integer")
        self.value = value

class Person:
    def __init__(self):
        self.age = PositiveInteger()

# Usage
p = Person()
p.age = 30  # Valid
print(p.age)  # Output: 30

try:
    p.age = -5  # Raises ValueError
except ValueError as e:
    print(e)  # Output: Must be a positive integer

print(p.age)  # Output: 30


30
-5


In [8]:
class PositiveInteger:
    def __init__(self):
        self.value = 0

    def __get__(self, instance, owner):
        return self.value

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Must be a positive integer")
        self.value = value

class Person:
    def __init__(self):
        self._age = PositiveInteger()  # Use an instance variable to store the descriptor

    @property
    def age(self):
        return self._age.__get__(self, Person)

    @age.setter
    def age(self, value):
        self._age.__set__(self, value)

# Usage
p = Person()
p.age = 30  # Valid
print(p.age)  # Output: 30

try:
    p.age = -5  # Raises ValueError
except ValueError as e:
    print(e)  # Output: Must be a positive integer

print(p.age)  # Output: 30

30
Must be a positive integer
30
